In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('foodbank_data_with_regions.csv')

In [3]:
import json
fileObject = open("needs.json", "r")
jsonContent = fileObject.read()
aList = json.loads(jsonContent)

In [10]:
list_dict_needs = {}
for i in range(len(aList)):
    list_dict_needs[aList[i]['foodbank']['name']] = []
for i in range(len(aList)):
    list_dict_needs[aList[i]['foodbank']['name']] = list_dict_needs[aList[i]['foodbank']['name']] + aList[i]['needs'].split('\r\n')

In [15]:
aList[0]['needs'].split('\r\n')

['UHT Milk (Full Fat Or Semi Skimmed)',
 'Jam',
 'Cereals',
 'Pasta Sauce',
 'Soup',
 'Sugar',
 'UHT Milk (Full Fat Or Semi Skimmed)',
 'Tinned Fruit',
 'Jam',
 'Shampoo',
 'Deodorant']

In [18]:
import numpy as np

In [19]:
df['needs'] = [list_dict_needs[df['name'].iloc[i]] if df['name'].iloc[i] in list_dict_needs.keys() else np.nan for i in range(df.shape[0])]

In [20]:
df.set_index('name', inplace=True)

In [21]:
df.loc['Caithness']

parliamentary_constituency                Caithness, Sutherland and Easter Ross
latt_long                                                   58.594524,-3.522128
network                                                          Trussell Trust
county                                                                 Scotland
region                                                                 Scotland
needs                         [Tinned Fruit (400gms), Jam/marmalade, Cracker...
Name: Caithness, dtype: object

In [22]:
df2.to_csv('foodbank_locations_and_needs.csv')

In [5]:
from selenium import webdriver
driver = webdriver.Chrome()


In [68]:
with open('tesco_list.txt','r') as f:
    place_list = f.readlines()

In [69]:
for i in range(len(place_list)):
    place_list[i] = place_list[i].replace('\n','').replace(' ','-').lower()

In [70]:
'bishops-stortford' in place_list

True

In [71]:
import time
my_dict = {}
for place in place_list:
    driver.get(f'https://www.tesco.com/store-locator/{place}')
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    tag = 'address'
    attributes ={'class':'Address-content'}
    table_soup = soup.find_all(tag, attributes)
    my_list = []
    for i in range(len(table_soup)):
        address_soup = table_soup[i].find_all('span')
        my_string = ' '.join([j.text for j in address_soup])
        my_list.append(my_string)
    
    my_dict[f'{place}'] = my_list

In [81]:
import json
with open('tesco_dict.json','w') as f:
    json.dump(my_dict, f)

In [23]:
a = table_soup[1].find_all('span')
string = ' '.join([i.text for i in a])

NameError: name 'table_soup' is not defined

In [88]:
place = 'london'
driver.get(f'https://www.catalogueoffers.co.uk/stores/morrisons/locations/{place}')
soup = BeautifulSoup(driver.page_source, 'html.parser')
tag = 'div'
attributes ={'class':'location-name'}
table_soup = soup.find_all(tag, attributes)
for i in table_soup:
    address = i.text
    address.replace('\n','').lstrip()


264 Walworth Road SE171JE London                        
399 Edgware Road NW90AS London                        
Aylesham Centre, Rye Lane SE155EW London                        
Butterfly Walk SE58RR London                        
The Grove E151HP London                        


In [ ]:
my_list = []
for i in range(len(table_soup)):
    address_soup = table_soup[i].find_all('span')
    my_string = ' '.join([j.text for j in address_soup])
    my_list.append(my_string)
    
my_dict[f'{place}'] = my_list

In [95]:
for shop in ['asda','sainsbury-s','morrisons']:

    with open(f'{shop}.txt','r') as f:
        place_list = f.readlines()
    
    for i in range(len(place_list)):
        place_list[i] = place_list[i].replace('\n','').replace(' ','-').replace("'",'-').lower()

    my_dict = {}
    for place in place_list:
        driver.get(f'https://www.catalogueoffers.co.uk/stores/{shop}/locations/{place}')
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        tag = 'div'
        attributes ={'class':'location-name'}
        table_soup = soup.find_all(tag, attributes)

        my_list=[]
        for i in table_soup:
            address = i.text
            my_list.append(address.replace('\n','').lstrip().rstrip())
        my_dict[f'{place}'] = my_list

    with open(f'{shop}_dict.json','w') as f:
        json.dump(my_dict, f)

In [1]:
import json
with open('sainsbury-s_dict.json','r') as f:
   data = json.load(f)
for i in data:
    for j in range(len(data[i])):
        indexing = data[i][j].rindex(' ') - 3
        data[i][j] = data[i][j][:indexing]+' '+ data[i][j][indexing:]


In [12]:
import json
with open('asda_dict.json','r') as f:
    data = json.load(f)

for i in data:
    for j in range(len(data[i])):
        indexing = data[i][j].rindex(' ', 0, data[i][j].rindex(' ', 0, data[i][j].rindex(' ')))
        data[i][j] = data[i][j][indexing+1:]

with open('asda_dict2.json','w') as f:
   json.dump(data, f)


In [189]:
data['london'][0][:data['london'][0].index(' ')+1] + data['london'][0][data['london'][0].rindex(' ')+1:] 

'N9 London'

In [13]:
import json
from geopy.geocoders import Nominatim
with open('asda_dict2.json','r') as f:
   data = json.load(f)

geolocator = Nominatim(user_agent="jm-test")
latlong_dict = {}
failed_list  = []
for i in data:
    location = []
    for j in range(len(data[i])):
        try:
            location.append((geolocator.geocode(data[i][j][:data[i][j].rindex(' ')]).latitude, geolocator.geocode(data[i][j][:data[i][j].rindex(' ')]).longitude))
        except:
            failed_list.append([i,data[i][j]])
    latlong_dict[i] = location


aberdare
aberdeen
accrington
1
alloa
altrincham
andover
anfield
antrim
arbroath
ardrossan
armadale
ashford-kent
ashton-under-lyne
aylesbury
ayr
ballyclare
bangor
barking-dagenham
barnsley
barnstaple
barrhead
barrow-in-furness
barry
basildon
basingstoke
batley
bebington
belfast
2
belvedere
beverley
bexley
bideford
biggleswade
billingham
birkenhead
birmingham
bishop-auckland
blaby
blackburn
blackpool
blackwood
blantyre
bletchley
bliston
bodmin
bolton
bootle
borehamwood
borrowash
boston
bournemouth
bradford
brampton
bridgend
bridgwater
brierley-hill
brighton
bristol
broadstairs
bromsgrove
burnley
burton-upon-trent
bury
bury-st-edmunds
caerphilly
caldicot
cambridge
cannock
canterbury
cardiff
carlisle
carterton
castleford
catshill
chatham
cheadle-chesire
cheadle-staffordshire
chelmsford
cheltenham
chester
chesterfield
chichester
chorley
clacton-on-sea
clevedon
clydebank
coalville
coatbridge
colchester
coleraine
3
colne
conisbrough
cookstown
4
corby
costessey
coventry
cramlington
crawley
cre

In [ ]:
import json
from geopy.geocoders import Nominatim
with open('tesco_dict.json','r') as f:
   data = json.load(f)

for i in data:
    for j in range(len(data[i])):
        indexing = data[i][j].rindex(' ', 0, data[i][j].rindex(' '))
        data[i][j] = data[i][j][indexing+1:]


In [2]:
data['london'][0]

'E6 7ER'

In [3]:

geolocator = Nominatim(user_agent="jm-test")
latlong_dict = {}
failed_list  = []
for i in data:
    print(i)
    location = []
    for j in range(len(data[i])):
        try:
            location.append((geolocator.geocode(data[i][j]).latitude, geolocator.geocode(data[i][j]).longitude))
        except:
            failed_list.append([i,data[i][j]])
            print(len(failed_list))
    latlong_dict[i] = location


abbots-langley
aberdare
aberdeen
aberdeenshire
abergavenny
abergele
abertawe
abertillery
aberystwyth
abingdon
accrington
addlestone
adlington
ainsdale
airdrie
alcester
aldeburgh
alderley-edge
aldershot
alfreton
allerton-bywater
alloa
alresford
alton
altrincham
ambleside
amersham
ammanford
andover
annan
antrim
arbroath
archway
ascot
ash-vale
1
ashby-de-la-zouch
ashford
ashtead
ashton-under-lyne
atherton
aviemore
axminster
aylesbury
aylesford
ayr
baildon
baillieston
baldock
ballymena
ballymoney
2
balsall-common
banbridge
banbury
banchory
banff
bangor
banstead
barking
barnet
barnham
barnsley
barnstaple
barrow-in-furness
barry
barton-upon-humber
basildon
basingstoke
bath
3
bathgate
batley
battle
battle-hill
beccles
beckenham
bedale
bedfont
bedford
bedminster
bedworth
beeston
belfast
4
5
6
7
8
bellshill
belper
benfleet
benllech
berkhamsted
berkshire
berwick-upon-tweed
bethesda
beverley
bewdley
bexhill-on-sea
bicester
bideford
billericay
billingham
billingshurst
bilston
bingley
birkenhead
bi

In [208]:
geolocator = Nominatim(user_agent="jm-test")
for i in data:
    if i[0] >= 'w':
        print(i)
        location = []
        for j in range(len(data[i])):
            try:
                location.append((geolocator.geocode(data[i][j]).latitude, geolocator.geocode(data[i][j]).longitude))
            except:
                failed_list.append([i,data[i][j]])
    latlong_dict[i] = location

wakefield
wallasey
wallsend
walsall
ware
warrington
washington
waterlooville
watford
wath-upon-dearne
wellington
wembley
west-malling
weston-super-mare
wheatley
whitehaven
widnes
wigan
winsford
wisbech
witham


In [ ]:
with open('asda_latlong.json','w') as f:
   json.dump(latlong_dict, f)

In [9]:
print(failed_list)

[['blaydon', 'NE21 5QE Blaydon'], ['brentford', 'TW8 0AB Brentford'], ['chiseldon', 'SN3 3TX Chiseldon'], ['eastbourne', 'BN23 6QL Eastbourne'], ['holyhead', 'LL65 2ZZ Holyhead'], ['leeds', 'LS10 2AU Leeds'], ['royal-leamington-spa', 'CV31 3NU Royal-Leamington-Spa'], ['st-helens', 'WA9 3DY St-Helens']]
